
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Training Models </h2>	

In [12]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

In [13]:
def train_test_split(df):
    train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)
    return train_df, val_df

#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.mean(pred == actual)
# Calculate the accuracy percentage of the predicted values
def separate_features(df):
    feature_df = df.iloc[:,2:]
    labels_df = df.iloc[:,1]
    return feature_df, labels_df

In [14]:
training_data = pd.read_hdf('training_data.h5')

In [15]:
# training_data.head()

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


In [16]:
train, val = train_test_split(training_data)
X_train, y_train = separate_features(train)
X_train.reset_index().drop(columns=['index'])
y_train.reset_index().drop(columns=['index'])
y_train = y_train.astype('int')

X_val, y_val = separate_features(val)
X_val.reset_index().drop(columns=['index'])
y_val.reset_index().drop(columns=['index'])
y_val = y_val.astype('int')

In [17]:
import warnings
warnings.filterwarnings("ignore")

def cross_val(model, k, feature_data, label_data):
    
    kf = sklearn.model_selection.KFold(n_splits = k)
    kf.get_n_splits(feature_data)
    
    accuracy_scores = np.empty([k, 1])
    model_params = [{} for i in range(feature_data.shape[1])]
        
    for i, (train_index, test_index) in enumerate(kf.split(feature_data)):
        model.fit(feature_data.iloc[train_index], label_data.iloc[train_index])
        label_predict = model.predict(feature_data.iloc[test_index])
        accuracy_scores[i] = accuracy(label_data.iloc[test_index], label_predict)
        model_params[i] = model.get_params()
        print(accuracy_scores[i])
    
    return model_params, accuracy_scores

Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression(random_state=0, penalty='l2', C=1)
params = {'C': [1e-1, 1, 10, 100]}
lreg = GridSearchCV(lreg, params, cv=5)
model_params, accuracy_scores = cross_val(lreg, 5, X_train, y_train)

[ 0.34166667]
[ 0.30416667]
[ 0.37916667]
[ 0.32916667]
[ 0.28333333]


K-nearest Neighbors

In [19]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_train, y_train);

Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

rand_forest = RandomForestClassifier(
    random_state=0, 
    n_estimators=25, 
    max_depth=5, 
    bootstrap=True
)
params = {
    'n_estimators': [25,75,150],
    'max_depth': [10,20,80]
}
grid_rand_forest = GridSearchCV(rand_forest, params, cv=5)
model_params, accuracy_scores = cross_val(grid_rand_forest, 5, X_train, y_train)

[ 0.34166667]
[ 0.34166667]
[ 0.38333333]
[ 0.35833333]
[ 0.2625]


In [21]:
rand_forest = RandomForestClassifier(
    random_state=0, 
    n_estimators=100, 
    max_depth=20, 
    bootstrap=True
)

cross_val(rand_forest, 5, X_train, y_train)
y_hat = rand_forest.predict(X_val)
accuracy(y_hat, y_val)

[ 0.3625]
[ 0.31666667]
[ 0.39166667]
[ 0.34583333]
[ 0.27083333]


0.37873754152823919

Support Vector Machine

In [22]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

svm = OneVsOneClassifier(LinearSVC(random_state=0))
svm.fit(X_train, y_train);

In [92]:
models = [lreg, neigh, rand_forest, svm]
names = ['Logistic Regression', 'K-NN', 'Random Forest', 'SVM']
max_name_length = len(max(names, key=len))

for model, name in zip(models, names):
    y_hat = model.predict(X_val)
    acc = accuracy(y_hat, y_val)
    print('{:19s} Accuracy: {}'.format(name, acc))

Logistic Regression Accuracy: 0.39867109634551495
K-NN                Accuracy: 0.26245847176079734
Random Forest       Accuracy: 0.3953488372093023
SVM                 Accuracy: 0.22591362126245848
